In [1]:
import numpy as np
import pandas as pd

In [2]:
mu_0=np.pi*4e-7         #真空磁导率
I=500                #电流设定值（A）
R=0.05              #阵列圆半径
M=np.arange(0,0.009,0.001)  #初始偏移距离(m)
alpha1=np.arange(0,23,0.5)            #初始偏移角度（deg)

In [11]:
pd.set_option("display.max_columns", 16)
np.set_printoptions(precision = 16)
# pd.set_option('precision', 16)

In [12]:
def B_detect(I,R,M,alpha):
    R1=np.sqrt(R**2+M**2-2*R*M*np.cos(alpha*np.pi/180))
    R2=np.sqrt(R**2+M**2-2*R*M*np.cos(0.5*np.pi-alpha*np.pi/180))
    R3=np.sqrt(R**2+M**2-2*R*M*np.cos(np.pi-alpha*np.pi/180))
    R4=np.sqrt(R**2+M**2-2*R*M*np.cos(0.5*np.pi+alpha*np.pi/180))
    B1=mu_0*I/2/np.pi/R1
    B2=mu_0*I/2/np.pi/R2
    B3=mu_0*I/2/np.pi/R3
    B4=mu_0*I/2/np.pi/R4
    return(B1,B2,B3,B4)

In [13]:
df__=pd.DataFrame({'M':[],'alpha':[],'B1':[],'B2':[],'B3':[],'B4':[]})
for M_ in M:
    for alpha1_ in alpha1:
        (B1,B2,B3,B4)=B_detect(I,R,M_,alpha1_)
        df__=df__.append({'M':M_,'alpha':alpha1_,'B1':B1,'B2':B2,'B3':B3,'B4':B4},ignore_index=True)
        
        

AttributeError: 'DataFrame' object has no attribute 'append'

In [30]:
#df__[df__['M']==0.008][df__['alpha']==22]

# 以上是二维

## 三维坐标用R0,theta,phi代表tmr单元的偏心距离，偏心角和垂直偏心角



### 四个tmr单元的初始坐标为【R（阵列圆半径），0,0】，【R（阵列圆半径），90,0】，【R（阵列圆半径），180,0】，【R（阵列圆半径），270,0】

In [1]:
import numpy as np
import pandas as pd

In [2]:
mu_0=np.pi*4e-7         #真空磁导率
I=500                #电流设定值（A）
R=0.05              #阵列圆半径(m)
#M=np.arange(0,0.0041,0.001)  #初始偏移距离(m)
#M=np.arange(0.0005,0.005,0.0005)  #初始偏移距离(m)，数据量缩小十倍，数据量为420万
M=np.arange(0,0.0045,0.0005)  #初始偏移距离(m)，数据量缩小十倍，数据量为420万
alpha1=np.arange(0,23,1)            #初始z轴偏移角度（deg)
alpha2=np.arange(0,45,1)  #初始x轴偏移角度（deg)
N=2   #气隙放大倍数

In [3]:
#定义旋转矩阵类
class Rotation:
    def __init__(self,I,R,R0=0.0,theta=0.0,phi=0.0,N=1.0):
        self.I=I
        self.tmr1=[R+R0,0.0,0.0]#四个阵列圆的初始坐标
        self.tmr2=[R0,R,0.0]
        self.tmr3=[R0,-R,0.0]
        self.tmr4=[-R+R0,0.0,0.0]
        self.R0=R0
        self.N=N  #N是气隙提供的放大倍数
        self.theta=theta#定义偏心距离，zx偏角
        self.phi=phi
        #计算阵列圆中心的位置变化 #阵列圆位置和法向矢量[0,0,1]均发生变化
        self.center=np.array([R0,0,0])
        #self.center_initial_0=np.array([R0,0,0])#阵列圆中心位置偏移后的坐标
        #self.center_initial=self.rotation_z(theta,[R0,0,0])#阵列圆中心位置偏移后的坐标
        #self.center=self.rotation_x(phi,self.rotation_z(theta,[R0,0,0]))#计算得到阵列圆中心位置偏移后的坐标
        self.penpendicular=self.rotation_x(phi,self.rotation_z(theta,[0,0,1]))#计算得到阵列圆法向偏移后的坐标
    def polar2xyz(self,p):#极坐标转化为xyz坐标,并进行坐标变换，以阵列圆中心点为[0,0,0],方便做旋转操作
        xyz=np.array([p[0]*np.cos(p[1]/180*np.pi)*np.cos(p[2]/180*np.pi),p[0]*np.sin(p[1]/180*np.pi)*np.cos(p[2]/180*np.pi),p[0]*np.sin(p[2]/180*np.pi)])
        return(xyz)
    def rotation_z(self,t,a):#z轴旋转矩阵
        rotationz=np.array([[np.cos(t/180*np.pi),-np.sin(t/180*np.pi),0],[np.sin(t/180*np.pi),np.cos(t/180*np.pi),0],[0,0,1]])
        return(np.dot(np.array(a),rotationz))
    def rotation_x(self,t,a):#x轴旋转矩阵
        rotationx=np.array([[1,0,0],[0,np.cos(t/180*np.pi),-np.sin(t/180*np.pi)],[0,np.sin(t/180*np.pi),np.cos(t/180*np.pi)]])
        return(np.dot(np.array(a),rotationx))
    def projection(self,a):#获得磁场和磁场在阵列圆切向的投影，a是tmr初始极坐标
        #tmr=self.rotation_x(self.phi,self.rotation_z(self.theta,np.array(a)-self.center_initial_0)+self.center_initial_0-self.center_initial)+self.center_initial_0
        tmr=self.rotation_x(self.phi,self.rotation_z(self.theta,np.array(a)- self.center))+self.center
        B1_direction=np.cross(np.array([0,0,1]),tmr/np.sqrt(tmr[0]**2+tmr[1]**2+tmr[2]**2)) #tmr位置坐标与电流方向【001】叉乘得到磁场方向
        B1_B=mu_0*self.I/2/np.pi/np.sqrt(tmr[0]**2+tmr[1]**2)#仅与xoy投影相关
        B1=B1_B*B1_direction#磁场方向乘以幅值得到探测点的磁场向量 
        #print(np.sqrt(tmr[0]**2+tmr[1]**2))
        B1_projection=np.cross(self.penpendicular,tmr-self.center)#计算阵列圆在tmr坐标处的切向方向
        B1_projection/=np.sqrt(B1_projection[0]**2+B1_projection[1]**2+B1_projection[2]**2)#转化成单位向量
        B1_=np.dot(B1,B1_projection)
        B2=self.N*np.dot(B1,B1_projection)#计算阵列圆切向方向磁场
        return(B1_,B2)
        
    def run(self):
        (B1_,B1)=self. projection(self.tmr1)#下划线的是原始磁场，无下划线的是测试得到的磁场
        (B2_,B2)=self. projection(self.tmr2)
        (B3_,B3)=self. projection(self.tmr3)
        (B4_,B4)=self. projection(self.tmr4)
        df={'R0':self.R0,'theta':self.theta,'phi':self.phi,'B1':B1,'B2':B2,'B3':B3,'B4':B4,'B1_':B1_,'B2_':B2_,'B3_':B3_,'B4_':B4_}
        return(df) 
        

In [4]:
#用df做输出问题在于不能实时保存
'''
df=pd.DataFrame({'R0':[],'theta':[],'phi':[],'B1':[],'B2':[],'B3':[],'B4':[],'B1_':[],'B2_':[],'B3_':[],'B4_':[]})
for M_ in M:
    for alpha1_ in alpha1:
        for alpha2_ in alpha2:
            A=Rotation(I,R,M_,alpha1_,alpha2_,N)
            df=df.append(A.run(),ignore_index=True)
            '''

"\ndf=pd.DataFrame({'R0':[],'theta':[],'phi':[],'B1':[],'B2':[],'B3':[],'B4':[],'B1_':[],'B2_':[],'B3_':[],'B4_':[]})\nfor M_ in M:\n    for alpha1_ in alpha1:\n        for alpha2_ in alpha2:\n            A=Rotation(I,R,M_,alpha1_,alpha2_,N)\n            df=df.append(A.run(),ignore_index=True)\n            "

In [5]:
Filepath='result.txt'
i=0

In [6]:
sum1=len(M)*len(alpha1)*len(alpha2)

In [7]:
A=Rotation(500,0.05,0.005,30,0,2)
df=A.run()
print(df)

{'R0': 0.005, 'theta': 30, 'phi': 0, 'B1': 0.0036734208060815693, 'B2': 0.0037837837837837837, 'B3': 0.004175824175824175, 'B4': 0.0043661711543264686, 'B1_': 0.0018367104030407846, 'B2_': 0.0018918918918918919, 'B3_': 0.0020879120879120877, 'B4_': 0.0021830855771632343}


In [8]:
import time

In [9]:
df_=pd.DataFrame({'R0(m)':[],'theta(deg)':[],'phi(deg)':[],'B1(T)':[],'B2(T)':[],'B3(T)':[],'B4(T)':[],'B1_(T)':[],'B2_(T)':[],'B3_(T)':[],'B4_(T)':[]})#清空

In [10]:
with open(Filepath,'w') as file_object:#新建实时输出数据文件
        file_object.write("by SLK:\nR0(m)\ttheta(deg)\tphi(deg)\tB1(T)\tB2(T)\tB3(T)\tB4(T)\tB1_(T)\tB2_(T)\tB3_(T)\tB4_(T)\n")

In [11]:
i=0
start=time.process_time()
start0=start
yx=True
for M_ in M:
    for alpha1_ in alpha1:
        for alpha2_ in alpha2:
            i+=1
            end = time.process_time()
            if end-start0>60:#每60s汇报一次进展，确保没崩溃
                start0=end
                print('完成进度'+str(i/sum1*100)+'%,时间为：'+str(end-start)+'s\t当前M为'+str(M_)+'m\n')
                if yx==False:#如果断了重做，就会显示恢复进度
                    print('正在恢复进度...\n')
            #if i>=j: #手动断点续算
            #自动断点续算功能，重复点不再计算    
            if i>=df_.shape[0] or (df_['R0(m)'][i-1]-M_>=0.00001 and df_['theta(deg)'][i-1]-alpha1_>=0.01 and df_['phi(deg)'][i-1]-alpha2_>=0.01):
                A=Rotation(I,R,M_,alpha1_,alpha2_,N)
                df=A.run()
                yx=True
                string=str(df['R0'])+'\t'+str(df['theta'])+'\t'+str(df['phi'])+'\t'+str(df['B1'])+'\t'+str(df['B2'])+'\t'+str(df['B3'])+'\t'+str(df['B4'])+'\t'+str(df['B1_'])+'\t'+str(df['B2_'])+'\t'+str(df['B3_'])+'\t'+str(df['B4_'])+'\n'
                with open(Filepath,'a') as file_object:#'a'表示可以修改，不覆盖
                    file_object.write(string) 
            else:
                yx=False

In [12]:
df_=pd.read_csv('result.txt',sep='\t',header=1)

In [13]:
j=df_.shape[0]

In [14]:
j

9315

In [15]:
df_['R0(m)'][j-1]

0.004

In [16]:
df_.iloc[-1]

R0(m)          0.004000
theta(deg)    22.000000
phi(deg)      44.000000
B1(T)          0.002759
B2(T)          0.003556
B3(T)          0.003935
B4(T)          0.003235
B1_(T)         0.001379
B2_(T)         0.001778
B3_(T)         0.001967
B4_(T)         0.001617
Name: 9314, dtype: float64

In [17]:
df_

,R0(m),theta(deg),phi(deg),B1(T),B2(T),B3(T),B4(T),B1_(T),B2_(T),B3_(T),B4_(T)
0,0.000,0,0,0.004000,0.004000,0.004000,0.004000,0.002000,0.002000,0.002000,0.002000
1,0.000,0,1,0.003999,0.004000,0.004000,0.003999,0.002000,0.002000,0.002000,0.002000
2,0.000,0,2,0.003998,0.004000,0.004000,0.003998,0.001999,0.002000,0.002000,0.001999
3,0.000,0,3,0.003995,0.004000,0.004000,0.003995,0.001997,0.002000,0.002000,0.001997
4,0.000,0,4,0.003990,0.004000,0.004000,0.003990,0.001995,0.002000,0.002000,0.001995
5,0.000,0,5,0.003985,0.004000,0.004000,0.003985,0.001992,0.002000,0.002000,0.001992
6,0.000,0,6,0.003978,0.004000,0.004000,0.003978,0.001989,0.002000,0.002000,0.001989
7,0.000,0,7,0.003970,0.004000,0.004000,0.003970,0.001985,0.002000,0.002000,0.001985
8,0.000,0,8,0.003961,0.004000,0.004000,0.003961,0.001981,0.002000,0.002000,0.001981
9,0.000,0,9,0.003951,0.004000,0.004000,0.003951,0.001975,0.002000,0.002000,0.001975


## 再造一个验证集！

In [18]:
import numpy as np
import pandas as pd

In [19]:
mu_0=np.pi*4e-7         #真空磁导率
I=500                #电流设定值（A）
R=0.05              #阵列圆半径(m)
#M=np.random.uniform(0.0,0.0400,27000)  #初始偏移距离(m)
#alpha1=np.random.uniform(0.0,23.0,27000)            #初始z轴偏移角度（deg)
#alpha2=np.random.uniform(0.0,45.0,27000)  #初始x轴偏移角度（deg)

#M=np.random.uniform(0.0005,0.0050,7000)  #初始偏移距离(m)
M=np.random.uniform(0,0.00450,7000)  #初始偏移距离(m)
alpha1=np.random.uniform(0.0,23.0,7000)            #初始z轴偏移角度（deg)
alpha2=np.random.uniform(0.0,45.0,7000)  #初始x轴偏移角度（deg)
N=2   #气隙放大倍数

In [20]:
import time

In [21]:
df2_=pd.DataFrame({'R0(m)':[],'theta(deg)':[],'phi(deg)':[],'B1(T)':[],'B2(T)':[],'B3(T)':[],'B4(T)':[],'B1_(T)':[],'B2_(T)':[],'B3_(T)':[],'B4_(T)':[]})#清空

In [22]:
with open('testset27000-2.txt','w') as file_object:#新建实时输出数据文件
        file_object.write("by SLK:\nR0(m)\ttheta(deg)\tphi(deg)\tB1(T)\tB2(T)\tB3(T)\tB4(T)\tB1_(T)\tB2_(T)\tB3_(T)\tB4_(T)\n")

In [23]:
sum1=len(M)

In [24]:
i=0
start=time.process_time()
start0=start
for M_ in M:
    i+=1
    end = time.process_time()
    if end-start0>10:#每10s汇报一次进展，确保没崩溃
        start0=end
        print('完成进度'+str(i/sum1*100)+'%,时间为：'+str(end-start)+'s\t当前M为'+str(M_)+'m\n')
    #if i>=j: #手动断点续算
    #自动断点续算功能，重复点不再计算    
    A=Rotation(I,R,M[i-1],alpha1[i-1],alpha2[i-1],N)
    df=A.run()
    yx=True
    string=str(df['R0'])+'\t'+str(df['theta'])+'\t'+str(df['phi'])+'\t'+str(df['B1'])+'\t'+str(df['B2'])+'\t'+str(df['B3'])+'\t'+str(df['B4'])+'\t'+str(df['B1_'])+'\t'+str(df['B2_'])+'\t'+str(df['B3_'])+'\t'+str(df['B4_'])+'\n'
    with open('testset27000-2.txt','a') as file_object:#'a'表示可以修改，不覆盖
         file_object.write(string) 
            

完成进度59.24285714285714%,时间为：10.015625s	当前M为0.0026755474260911644m

